In [9]:
# installing required SQL magic packages 
#!pip install sqlachemy
#!pip install ipython-sql
#!pip install pymysql

In [5]:
##import necessary libraries
import sqlite3
from sqlalchemy import create_engine
import pandas as pd


In [6]:
# connect to the sqlite database (it will be created since it doesn't exit)
%load_ext sql

engine = create_engine("sqlite:///bonga_ecommerce.db")
%config SQL.conn_name = 'engine'

conn = sqlite3.connect("bonga_ecommerce.db")
cursor = conn.cursor()

%sql sqlite:///bonga_ecommerce.db

# Create table  

In [7]:
# Data Definition Languages (DDL)
# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS products(
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price DECIMAL NOT NULL,
    category TEXT NOT NULL
)    
''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS customers(
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
)    
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders(
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)    
''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS orderitems(
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity product_id NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)    
''')







In [8]:
%%sql
SELECT name FROM sqlite_master WHERE type in ('table','view') AND name NOT LIKE 'sqlite_%' ORDER BY 1

 * sqlite:///bonga_ecommerce.db
Done.


name
customers
orderitems
orders
products


In [9]:
# Drop tables
cursor.execute('''
DROP TABLE IF EXISTS products;
''')

cursor.execute('''
DROP TABLE IF EXISTS customers;
''')

cursor.execute('''
DROP TABLE IF EXISTS orders;
''')

cursor.execute('''
DROP TABLE IF EXISTS ordersitems;
''')

In [90]:
#function to load the data into the database table
def load_data_to_table(csv_path, table_name):
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, conn, if_exists='append', index=False)

In [11]:
check = pd.read_csv('Order_items.csv')

check.head()

,order_item_id,order_id,product_id,quantity
0,1,1,34,2
1,2,1,653,1
2,3,1,427,3
3,4,2,830,3
4,5,2,886,4


In [12]:
find_duplicate = check.duplicated(subset=['order_item_id']).sum()

print(find_duplicate)

0


In [ ]:
# loading the dataset to our database table
load_data_to_table('Products.csv', 'products')
load_data_to_table('Customers.csv', 'customers')
load_data_to_table('Orders.csv', 'orders')
load_data_to_table('Order_items.csv', 'orderitems')  

# DATA MANIPULATION LANGUAGE

 ## INSERT
 Example1

# Adding a new product

In [ ]:
%%sql

INSERT INTO products (name, price, category) VALUES ('New gadget', 299.99, 'Electronics')

In [ ]:
%%sql

SELECT * FROM products; 

In [ ]:
cursor.execute("INSERT INTO products (name, price, category) VALUES (?,?,?)", ('New laptop', 399.99, 'Electronics'))

conn.commit()

In [ ]:
%%sql

SELECT * FROM products; 

# UPDATE

In [ ]:
%%sql

SELECT * FROM customers; 

In [ ]:
%%sql

UPDATE customers SET email = 'Customer1@example.com' WHERE name = 'Customer1';


In [ ]:
%%sql

SELECT * FROM customers; 

In [ ]:
%%sql

UPDATE customers SET name = 'Jane Doe' WHERE email = 'Customer2@gmail.com';

In [ ]:
%%sql

SELECT * FROM customers; 

# DELETE 

In [ ]:
%%sql

SELECT * FROM orders; 

In [ ]:
%%sql

    DELETE FROM orders WHERE order_id = 1; 

In [ ]:
%%sql

SELECT * FROM orders; 

In [ ]:
%%sql

    DELETE FROM products WHERE name = 'New gadget'; 

In [ ]:
%%sql

SELECT * FROM products; 

# DATA QUERY LANGUAGE (DQL)

In [ ]:
%%sql

SELECT *
FROM products; 

In [ ]:
%%sql

SELECT *
FROM products
LIMIT 100; 

In [ ]:
%%sql

SELECT *
FROM products
LIMIT 30; 

In [ ]:
%%sql

SELECT product_id, name
FROM products
LIMIT 100; 

In [ ]:
%%sql

SELECT product_id as 'ID', name as 'Product_Title'
FROM products
LIMIT 100; 

# LIKE  OPERATOR

In [ ]:
%%sql

SELECT *
FROM customers
WHERE email LIKE '%@gmail.com' 

In [ ]:
%%sql

SELECT *
FROM customers
WHERE email LIKE '@gmail.com%' 

In [ ]:
%%sql

SELECT *
FROM customers
WHERE email LIKE '%@gmail.com%' 

In [ ]:
%%sql

SELECT *
FROM products
WHERE name LIKE '%laptop%' 

In [ ]:
%%sql

INSERT INTO products (name, price, category) VALUES ('New gadget', 299.99, 'Electronics')

In [ ]:
%%sql

SELECT *
FROM products
WHERE name LIKE '%gadget%' 

# BETWEEN

In [ ]:
%%sql

SELECT *
FROM orders
WHERE order_date BETWEEN '2022-01-01' and '2022-12-31'



In [ ]:
%%sql

SELECT *
FROM products
WHERE price BETWEEN 100 and 200


In [ ]:
%%sql

SELECT count(*) as 'Total Product between price 100-200'
FROM products
WHERE price BETWEEN 100 and 200


# Apply Aggregate functions
Example 1: Calculate the total sale per category

In [32]:
%%sql

SELECT category, SUM(price * quantity) as 'total_sales'
FROM orderitems as op, products as p
WHERE op.product_id = p.product_id
GROUP BY category;


 * sqlite:///bonga_ecommerce.db
Done.


category,total_sales
Books,819475.77
Electronics,782752.77
Fashion,794716.45
Home & Kitchen,662919.49
Toys,716282.38


## Example:2  Calculate the highest selling price

In [39]:
%%sql

SELECT product_id, SUM(quantity) as 'total_quantity'
FROM orderitems
GROUP BY product_id
ORDER BY Total_quantity DESC
LIMIT 10;


 * sqlite:///bonga_ecommerce.db
Done.


product_id,total_quantity
79,41
439,40
130,38
921,33
128,33
106,33
939,32
919,32
798,31
551,31


In [41]:
%%sql

SELECT COUNT(*) as 'Total Electronics Product'
FROM products
WHERE category = 'Electronics';

 * sqlite:///bonga_ecommerce.db
Done.


Total Electronics Product
193


### HAVING clause

SELECT columns

FROM tables

WHERE boolean expression

GROUP BY columns

HAVING boolean expression





In [64]:
%%sql

SELECT p.category, COUNT(od.order_id) as 'orders_count'
FROM orders as od, products as p, orderitems as op
WHERE od.order_id = op.order_id AND
      op.product_id = p.product_id 
GROUP BY category
HAVING orders_count > 10;


 * sqlite:///bonga_ecommerce.db
Done.


category,orders_count
Books,1302
Electronics,1181
Fashion,1247
Home & Kitchen,1019
Toys,1251


Example 2: Average Categories with more than 10 sales

In [67]:
%%sql

SELECT category, AVG(price) as 'average_price'
FROM products
GROUP BY category
ORDER BY average_price DESC; 

 * sqlite:///bonga_ecommerce.db
Done.


category,average_price
Fashion,261.46653846153845
Electronics,256.1550259067357
Home & Kitchen,253.98537572254335
Books,245.67896226415095
Toys,235.41691588785048


 ### JOINS, UNION, CONCATENATION




## Inner Join
 Example: Products and order

In [71]:
%%sql

SELECT p.name, od.order_date
FROM products as p
INNER JOIN orderitems as op ON p.product_id = op.product_id 
INNER JOIN orders as od ON op.order_id = od.order_id 
     

 * sqlite:///bonga_ecommerce.db
Done.


name,order_date
Product 1,2013-02-13 23:54:35
Product 1,2014-09-06 13:05:38
Product 1,2022-02-03 06:32:24
Product 1,2023-08-10 19:15:59
Product 1,2023-08-21 06:01:01
Product 1,2012-11-14 16:43:08
Product 1,2013-08-24 04:21:06
Product 1,2012-11-16 00:30:15
Product 2,2013-01-07 21:20:26
Product 2,2023-11-09 12:31:25


In [73]:
%%sql

SELECT *
FROM customers as c
INNER JOIN orders as od ON c.customer_id = od.customer_id 


 * sqlite:///bonga_ecommerce.db
Done.


customer_id,name,email,order_id,customer_id_1,order_date
1,Customer1,Customer1@hotmail.com,1331,1,2018-08-13 02:38:21
2,Customer2,Customer2@gmail.com,26,2,2014-11-04 05:41:36
2,Customer2,Customer2@gmail.com,132,2,2016-02-17 07:12:21
2,Customer2,Customer2@gmail.com,723,2,2023-05-22 09:03:06
2,Customer2,Customer2@gmail.com,1290,2,2017-04-12 15:23:51
2,Customer2,Customer2@gmail.com,1573,2,2019-03-21 14:15:36
3,Customer3,Customer3@gmail.com,354,3,2013-07-02 12:58:46
3,Customer3,Customer3@gmail.com,622,3,2011-03-31 10:59:46
3,Customer3,Customer3@gmail.com,858,3,2019-04-14 13:43:44
3,Customer3,Customer3@gmail.com,1831,3,2020-06-12 02:27:38


## Left Join

In [76]:
%%sql

SELECT c.name, od.order_id
FROM customers as c
LEFT JOIN orders as od ON c.customer_id = od.customer_id 


 * sqlite:///bonga_ecommerce.db
Done.


name,order_id
Customer1,1331
Customer2,26
Customer2,132
Customer2,723
Customer2,1290
Customer2,1573
Customer3,354
Customer3,622
Customer3,858
Customer3,1831


## UNION

In [79]:
%%sql

SELECT name
FROM customers
UNION 
SELECT name
FROM products
LIMIT 5


 * sqlite:///bonga_ecommerce.db
Done.


name
Customer1
Customer10
Customer100
Customer101
Customer102


In [81]:
%%sql

SELECT name
FROM customers
UNION 
SELECT name
FROM products

 * sqlite:///bonga_ecommerce.db
Done.


name
Customer1
Customer10
Customer100
Customer101
Customer102
Customer103
Customer104
Customer105
Customer106
Customer107


## CONCATENATION

In [88]:
%%sql
SELECT name  || ' ' || email as 'full_details'
FROM customers;

 * sqlite:///bonga_ecommerce.db
Done.


full_details
Customer1 Customer1@hotmail.com
Customer2 Customer2@gmail.com
Customer3 Customer3@gmail.com
Customer4 Customer4@hotmail.com
Customer5 Customer5@yahoo.com
Customer6 Customer6@gmail.com
Customer7 Customer7@gmail.com
Customer8 Customer8@gmail.com
Customer9 Customer9@hotmail.com
Customer10 Customer10@gmail.com
